In [39]:
from bs4 import BeautifulSoup
import requests as req
from datetime import datetime, date, timedelta
from crawler import Crawlin
import matplotlib.pyplot as plt
import matplotlib.dates

class NaverNews (Crawlin):

    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)
        self.data['title'] = "Crawlius"
        self.data['time'] = datetime.now().strftime('%Y-%m-%d %h')
        self.data['keywords'] = ['이태원']
        self.data['maxpages'] = 0
        self.data['total_articles'] = 0
        self.data['total_comments'] = 0

        self.sorting = 0
        self.silent = False

        d1 = timedelta(days=1)
        ts = date(2022, 10, 1)
        self.date_range = []

        self.x = []
        self.ya = []
        self.yc = []

        while ts <= date(2022, 10, 31):
            self.x.append(ts)
            self.date_range.append(ts.strftime("%Y.%m.%d"))
            ts = ts + d1             

    def crawlin(self)->list:

        for keyword in self.data['keywords']:

            res = []
            print(f"Searching Keyword: {keyword}")
            
            for date in self.date_range:

                page = 0
                last_page = -1

                acount = 0
                ccount = 0

                print(f"Searching {date}")

                while True:

                    if last_page == -1:
                        print(f"Searching page {page+1}...           \r")
                    else:
                        print(f"Searching page {page+1}/{last_page+1}... \r")

                    base_url = f"https://search.naver.com/search.naver?where=news&sm=tab_pge&query={keyword}&sort={self.sorting}&photo=0&field=0&pd=3&ds={date}&de={date}&cluster_rank=267&mynews=0&office_type=0&office_section_code=0&news_office_checked=&start={page}1"
                    
                    try:
                        src_page = req.get(base_url)
                    except:
                        print(f"Cannot get HTML page from page {page+1}")
                        continue

                    chicken_soup = BeautifulSoup(src_page.text, 'html.parser')
                    naver_news_urls = chicken_soup.find_all('a', "info", string="네이버뉴스")

                    for tit in naver_news_urls:
                        print(tit.parent.parent.parent.find('a', {'class':['news_tit', 'sub_tit']})['title'])

                    for addr in naver_news_urls:
                        com_page = req.get(addr['href'], headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.56'})
                        corn_soup = BeautifulSoup(com_page.text, 'html.parser')
                    
                        try: comments = corn_soup.find('span', 'u_cbox_count').text
                        except:
                            print("Cannot read comments") 
                            continue

                        self.data['total_articles'] += 1
                        self.data['total_comments'] += comments

                        acount += 1
                        ccount += int(comments)

                    last_page = self.check_last_page(chicken_soup)

                    if page == last_page or page == self.data['maxpages'] - 1:
                        break

                    page += 1

                print(acount)
                print(ccount)
                self.ya.append(acount)
                self.yc.append(ccount)

            print(f"Searching Keyword Completed: {keyword} ({len(naver_news_urls)} found)")

        return 

    def check_last_page(self, soup:BeautifulSoup)->int:
        if soup.find_all('a', {'class':'btn_next'}, href=True):
            return -1
        else:
            btns = soup.find_all('a', {'class':'btn'})
            return int(btns[-1].text) - 1

    def draw(self):
        xd = matplotlib.dates.date2num(self.x)
        fig = plt.figure()
        plt.plot_date(xd, self.ya, 'b-')
        plt.plot_date(xd, self.yc, 'g-')
        fig.autofmt_xdate()
        plt.show()
        plt.imsave('graph.png')

nn = NaverNews()
nn.crawlin()
nn.draw()

Searching Keyword: 이태원
Searching 2022.10.01
Searching page 1...           
코람코, 이태원 크라운호텔→주상복합시설로 개발 나서
돈스파이크, 마약 전과 '대마초'였다…2008~9년 관련 범행 20차례
“살 쭉쭉 빠질텐데”...마약 유튜버 놀라게 한 돈스파이크
가호, 메타버스 세계 만난다! ‘XR 온라인 콘서트’ 개최
쏟아지는 여행 예능…절친부터 가족까지 '찰떡 호흡' 출연진
“MZ세대 오감만족”… 유통업계 핫플로 거듭난 레스토랑 눈길
[영상] 일본 유명 야구선수, 우승 소감으로 한국말 "사랑해요"
'본캐 매력에 푹…꿀노잼도 좋아' 쏟아지는 배우들 여행예능 [N초점]
Cannot read comments
Cannot read comments
Cannot read comments
Cannot read comments
Cannot read comments
Cannot read comments
Cannot read comments
Cannot read comments
Searching page 2...           
'본캐 매력에 푹…꿀노잼도 좋아' 쏟아지는 배우들 여행예능 [N초점]
가호, 메타버스 세계 만난다! ‘XR 온라인 콘서트’ 개최
자살노인 추모·동물권리장전 행진…시월 첫날, 각종 집회
[박정규의 작살]이재명을 스타로 만든 3인은 누굴까…박근혜·홍준표·이만희
[오늘의 주요일정] 사회(10월1일 토요일)
Cannot read comments
Cannot read comments
Cannot read comments
Cannot read comments
Cannot read comments
0
0
Searching 2022.10.02
Searching page 1...           
모든 집이 용산공원 뷰…이태원 크라운호텔 부지에 고급 주택
광화문에 뜬 '이태원 클라쓰' 포차…K의 모든 것 즐긴다
드라마 이태원 클라스 팝업스토어에서 즐기는 그라피티
"새벽 작업실에서…" 돈스파이크, 12년 전 